In [1]:
try:
    from scikeras.wrappers import KerasRegressor                     
except ImportError:
    !pip install scikeras
    from scikeras.wrappers import KerasRegressor
    
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn
from sklearn.pipeline import Pipeline # for setting up a pre-processing / tuning pipeline.
from sklearn.preprocessing import RobustScaler # Here, we are going to normalize inputs (the ML Pipeline framework from sklearn can implement this.)

# So, we are going back to the Boston Housing data here.
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = (boston_housing.load_data())

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
57026/57026 [==============================] - 0s 0us/step


#*Grid Search CV With Keras Model*

In [3]:
# Make sure you set your custom parameters for training as arguments in your model creation function.
def create_model(loss="mean_squared_error",optimizer="sgd",activation="relu",units=100,numLayers=2, batch_size=10):
    
    # I beleve that you need to explicitly declare an input layer for the scikeras wrapper to work... 
    model = keras.Sequential([
        layers.Input(train_data.shape[1]), 
        layers.Dense(units, activation="relu")             
    ])

    if numLayers == 2:
        model.add(layers.Dense(units, activation="relu"))

    model.add(layers.Dense(1, activation=activation))

    model.compile(loss=loss,optimizer=optimizer, metrics=['mse'])
    return model

# You also need to specify the 'custom' parameters here that you want to add, for them to show up as a trainable parameter in GridSearchCV.
regf = KerasRegressor(model=create_model, loss="mean_squared_error", optimizer="adam", activation="relu", units=100, numLayers=2, batch_size=10, verbose=0)

# Note you can also do a grid search over an sklearn pipeline, so you can search over diferent types of data pre-processing approaches too!
#ml_pipeline = Pipeline([("Normalize_with_centering", RobustScaler()), ("Model", regf)])

# Here are the configurable parameters we can now search over for either object. 
print(regf.get_params().keys())
#print(ml_pipeline.get_params().keys())

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'activation', 'units', 'numLayers'])


And this is how I would invoke my grid search... 

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

# Because we are creating the models but are not compiling them yet (we will let the grid fit compile the models on the fly),
# this will produce a bunch of warnings. I'm just suppressing the warnings. 
#import logging, os
#logging.disable(logging.WARNING)
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

params = {
    "numLayers": [1,2],
    "units": [10,50,100,150],
    "activation": ['relu','selu',None],
    "batch_size": [25,50],
    "epochs": [10,20,30]
}

params_pipe = {
    "Model__numLayers": [1,2],
    "Model__units": [10,100,500],
    "Model__activation": ['relu','selu',None],
    "Model__batch_size": [25,50],
    "Model__epochs":[10,20,30]
}

grid = GridSearchCV(regf, params, scoring='neg_mean_absolute_error',verbose=0,cv=10)
#grid = GridSearchCV(ml_pipeline, params_pipe, scoring='neg_mean_absolute_error',verbose=0,cv=10)

grid.fit(train_data, train_targets)

I can then extract the parameters that yielded the top performance... 

In [5]:
print(f"Best Score  : {grid.best_score_}")
print(f"Best Params : {grid.best_params_}")

Best Score  : -22.398225308641976
Best Params : {'activation': 'relu', 'numLayers': 1, 'units': 10}


Finally, a little function that looks at pairs of parameter values, and the associated model performance, holding all other parameters to their ideal values. 